In [ ]:
import requests
import json
import pandas as pd
import plotly.express as px

##  Lets Get some Crypto Prices !

## Function to fetch current price of a cryptocurrency


For reference on this API 
https://developers.cryptocompare.com/

In [ ]:
def get_current_price(crypto: str, currency: str = 'USD'):
    url = f'https://min-api.cryptocompare.com/data/price?fsym={crypto}&tsyms={currency}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code} - {response.text}"

In [ ]:
btc_price = get_current_price('BTC', 'USD')
print(btc_price)

In [ ]:
# Lets print the price but not as a Json
print(btc_price['USD'])


In [ ]:
print(f"Current price of BTC is: {btc_price['USD']} USD")

In [ ]:
# Lets format the price 
print(f"Current price of BTC is: {btc_price['USD']:,.2f} USD")

## Function to fetch historical data for a cryptocurrency



In [ ]:
def get_historical_data(crypto: str, currency: str = 'USD', limit: int = 100):
    url = f'https://min-api.cryptocompare.com/data/v2/histoday?fsym={crypto}&tsym={currency}&limit={limit}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code} - {response.text}"

In [ ]:
print("\nFetching historical price data for Ethereum...")
eth_historical = get_historical_data('ETH', 'USD')
print(json.dumps(eth_historical, indent=4))

In [ ]:
eth_historical

In [ ]:
eth_hist_data = eth_historical['Data']['Data']
eth_hist_data

### Create a dataframe from the historical data


In [ ]:
eth_hist_df = pd.DataFrame(eth_hist_data)
eth_hist_df 

### Convert the time to a readable format


In [ ]:
eth_hist_df['time'] = pd.to_datetime(eth_hist_df['time'], unit='s')
eth_hist_df

### Plot the historical data


In [ ]:
fig = px.line(eth_hist_df, x='time', y='close', title='ETH (Ethereum) Historical Price')
fig.show()

## Function to fetch top cryptocurrencies by market cap


In [ ]:

def get_top_cryptos(limit=20):
    url = "https://min-api.cryptocompare.com/data/top/mktcapfull"
    params = {
        "limit": limit,
        "page": 0,
        "tsym": "USD",
        "e": "CCCAGG"
    }
    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json()
    return pd.json_normalize(data["Data"])

In [ ]:
top_cryptos_df = get_top_cryptos(20)
top_cryptos_df

In [ ]:
# Lets just get the columns for the coin name and market cap
top_cryptos_df[['CoinInfo.FullName', 'RAW.USD.MKTCAP']]

In [ ]:
# make a bar chart of the top 20 cryptocurrencies by market cap
fig = px.bar(top_cryptos_df, x='CoinInfo.FullName', y='RAW.USD.MKTCAP', 
             title='Top 20 Cryptocurrencies by Market Cap',
             labels={'CoinInfo.FullName': 'Cryptocurrency', 'RAW.USD.MKTCAP': 'Market Cap (USD)'},
             text='RAW.USD.MKTCAP')
fig.update_traces(texttemplate='$%{text:,.0f}', textposition='outside')
fig.update_layout(yaxis_tickformat='$,.0f')
fig.show()

## Let's look at WLFI 

In [ ]:
# Lets get historical data for WLFI as we did for ETH above 
wlfi_historical = get_historical_data('WLFI', 'USD')
wlfi_hist_data = wlfi_historical['Data']['Data']
wlfi_hist_df = pd.DataFrame(wlfi_hist_data)
wlfi_hist_df['time'] = pd.to_datetime(wlfi_hist_df['time'], unit='s')   
wlfi_hist_df

In [ ]:
# plot the price for WLFI over the past year
fig = px.line(wlfi_hist_df, x='time', y='close', title='WLFI (World Liberty Financial) Historical Price')
fig.update_layout(yaxis_tickformat='$,.2f')
fig.show()

In [ ]:
import re

# Grab the WLFI row
wlfi_row = top_cryptos_df[top_cryptos_df["CoinInfo.Name"] == "WLFI"].iloc[0]

# Extract DISPLAY supply (string)
supply_str = wlfi_row["DISPLAY.USD.SUPPLY"]

# Keep only digits
wlfi_supply = int(re.sub(r"[^\d]", "", supply_str))

wlfi_supply

In [ ]:
wlfi_hist_df = wlfi_hist_df.copy()
wlfi_hist_df["marketcap"] = wlfi_hist_df["close"] * wlfi_supply

In [ ]:
# Plot WLFI Market Cap over time
fig = px.line(wlfi_hist_df, x='time', y='marketcap', title='WLFI (World Liberty Financial) Historical Market Cap')
fig.update_layout(yaxis_tickformat='$,.0f')
fig.show()